In [72]:
import pandas as pd
import seaborn as sns
import sklearn.cluster
import random

In [173]:
df_votants = pd.read_csv('database_deputes.csv')
df_votes = pd.read_csv('database_votes2.csv')

On transforme la date de naissance en âge en 2021. 

In [110]:
df_votes['age'] = 2021 - pd.to_numeric(df_votes['date_naissance'].str[:4])

On veut comparer tous les votants pour tous les scrutins et trouver ceux qui sont le plus proches. On crée une table dans laquelle idVotant est en colonne et chaque scrutin est en colonne. 

In [296]:
copy = df_votes.copy()

In [297]:
copy = copy.pivot(index = 'idVotant', columns = 'idScrutin', values = 'vote')

On supprime les lignes qui ont trop de NaN. On veut un nombre minimal de vote par votant pour que le résultat soit interprétable. 

In [298]:
#Nombre de scrutin par votant 
df_votes.groupby('idVotant')['idScrutin'].count().describe()

count     575.000000
mean      505.210435
std       289.414745
min         5.000000
25%       294.000000
50%       469.000000
75%       678.000000
max      2165.000000
Name: idScrutin, dtype: float64

Il y a en moyenne 505 scrutin par votant. L'écart type est de 289 scrutins. 
On ne garde dans un premier temps que les votants qui ont plus de q1 votes. On pourra affiner ce choix par la suite. Cad on veut un nombre minimal de nonNan de 575 - 290 = 285. 

In [299]:
copy = copy.dropna(thresh = 285) 
copy

idScrutin,0,1,2,3,4,5,6,7,8,9,...,3107,3108,3109,3110,3111,3112,3113,3114,3115,3116
idVotant,,,,,,,,,,,,,,,,,,,,,
PA1008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Pour,NaN,NaN,NaN,Pour
PA1012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pour,NaN,NaN
PA1198,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pour,NaN,...,NaN,NaN,NaN,NaN,NaN,Pour,NaN,NaN,NaN,NaN
PA1327,NaN,Contre,NaN,Pour,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,Pour,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PA1592,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pour,NaN,...,Pour,NaN,Pour,NaN,NaN,NaN,NaN,NaN,Pour,Pour
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PA724827,NaN,NaN,NaN,Contre,NaN,Pour,Contre,NaN,NaN,NaN,...,NaN,NaN,NaN,Pour,NaN,NaN,NaN,NaN,Contre,NaN
PA748954,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Contre,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pour
PA760658,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


On change les NaN pour mettre la valeur majoritaire de chaque scrutin. 

In [300]:
#un peu à l'arrache, il faudrait faire ça de manière plus jolie pcq là on sort une liste sans index 
#et on la rebalance dans le DF
#a priori pas de souci pcq le DF copy est trié mais bon c'est pas très clean
maj = []
for scrutin in range(3117):
    votemaj = pd.DataFrame(copy[scrutin].value_counts())[scrutin].idxmax()
    maj.append(votemaj)

In [301]:
copy.loc['vote_majoritaire'] = maj

In [302]:
copy = copy.fillna(copy.loc['vote_majoritaire'])

On fait un kmeans. 
Pb: est-ce que ça peut se faire avec des strings comme valeurs ? Ca a pas l'air de marcher... Ou est-ce qu'on doit tout convertir en float avec par exemple pour = 1, contre = -1, non votant = 0 ? Mais alors ça va faire des trucs bizarres si jamais il y a des calculs de moyenne non ? 

In [310]:
#je crois c'est cassé oupsie je sais pas comment ça marche 


import pandas as pd
from sklearn.datasets.samples_generator import make_blobs
from sklearn.cluster import KMeans

X, _ = make_blobs(n_samples=10, centers=5, n_features=3117)

df = copy

kmeans = KMeans(n_clusters=5)

df['Cluster'] = kmeans.fit_predict(df)

print(df.head())

idScrutin  0  1  2  3  4  5  6  7  8  9  ...  3108  3109  3110  3111  3112  \
idVotant                                 ...                                 
PA1008     1 -1 -1 -1 -1  1 -1  1 -1 -1  ...    -1    -1     1    -1     1   
PA1012     1 -1 -1 -1 -1  1 -1  1 -1 -1  ...    -1    -1     1    -1    -1   
PA1198     1 -1 -1 -1 -1  1 -1  1  1 -1  ...    -1    -1     1    -1     1   
PA1327     1 -1 -1  1 -1  1 -1  1 -1 -1  ...    -1     1     1    -1    -1   
PA1592     1 -1 -1 -1 -1  1 -1  1  1 -1  ...    -1     1     1    -1    -1   

idScrutin  3113  3114  3115  3116  Cluster  
idVotant                                    
PA1008        1    -1    -1     1        3  
PA1012        1     1    -1    -1        1  
PA1198        1    -1    -1    -1        1  
PA1327        1    -1    -1    -1        1  
PA1592        1    -1     1     1        1  

[5 rows x 3118 columns]


In [304]:
for scrutin in range(3117):
    copy[scrutin] = copy[scrutin].apply({'Pour':1, 'Contre':-1, 'Non-votant':0}.get)

In [305]:
copy

idScrutin,0,1,2,3,4,5,6,7,8,9,...,3107,3108,3109,3110,3111,3112,3113,3114,3115,3116
idVotant,,,,,,,,,,,,,,,,,,,,,
PA1008,1,-1,-1,-1,-1,1,-1,1,-1,-1,...,-1,-1,-1,1,-1,1,1,-1,-1,1
PA1012,1,-1,-1,-1,-1,1,-1,1,-1,-1,...,-1,-1,-1,1,-1,-1,1,1,-1,-1
PA1198,1,-1,-1,-1,-1,1,-1,1,1,-1,...,-1,-1,-1,1,-1,1,1,-1,-1,-1
PA1327,1,-1,-1,1,-1,1,-1,1,-1,-1,...,-1,-1,1,1,-1,-1,1,-1,-1,-1
PA1592,1,-1,-1,-1,-1,1,-1,1,1,-1,...,1,-1,1,1,-1,-1,1,-1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PA748954,1,-1,-1,-1,-1,1,-1,1,-1,-1,...,-1,-1,-1,1,-1,-1,1,-1,-1,1
PA760658,1,-1,-1,-1,-1,1,-1,1,-1,-1,...,-1,-1,-1,1,-1,-1,1,-1,-1,-1
PA856,1,-1,-1,-1,-1,1,-1,1,-1,-1,...,-1,-1,-1,1,-1,-1,1,-1,-1,-1
